<a href="https://colab.research.google.com/github/SRARNAB7/HDS_5230_07_Arnab/blob/main/Week%2009/Week_09_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## Import Modules
import os
import sys
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from patsy import dmatrices
from sklearn.metrics import confusion_matrix
import sklearn
from sklearn import datasets

In [2]:
## Set default figure size to be larger
## this may only work in matplotlib 2.0+!
matplotlib.rcParams['figure.figsize'] = [10.0,6.0]
## Enable multiple outputs from jupyter cells
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
## Get Version information
print(sys.version)
print("Pandas version: {0}".format(pd.__version__))
print("Matplotlib version: {0}".format(matplotlib.__version__))
print("Numpy version: {0}".format(np.__version__))
print("SciKitLearn version: {0}".format(sklearn.__version__))

3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0]
Pandas version: 2.2.2
Matplotlib version: 3.10.0
Numpy version: 2.0.2
SciKitLearn version: 1.6.1


Check your working directory
Set your working directory to make paths easier :)

In [4]:
# Working Directory
import os
print("My working directory:\n" + os.getcwd())
# Set Working Directory
os.chdir(".")
print("My new working directory:\n" + os.getcwd())

My working directory:
/content
My new working directory:
/content


Patient Mortality Dataset
We will use a dataset with a binary outcome of mortality as a motivating example.

This is a dataset of patients demographics and disease status, with mortality indicated. The dataset is here:

data\healthcare\patientAnalyticFile.csv

In practice, you most likely would have created a dataset like this from multiple other files after cleaning, reshaping, and joining them.

You can generalize this setup to any situation with a binary outcome, such as estimating the probability of a customer filing a warranty claim, or the probability of a transaction being fraudulent.

We will first import this dataset and examine the potential variables to use in our classification algorithm.

In [5]:
## Set print limits
pd.options.display.max_rows = 10
## Import Data
df_patient = \
 pd.read_csv('./PatientAnalyticFile.csv')
df_patient

,PatientID,DateOfBirth,Gender,Race,Myocardial_infarction,Congestive_heart_failure,Peripheral_vascular_disease,Stroke,Dementia,Pulmonary,...,Metastatic_solid_tumour,HIV,Obesity,Depression,Hypertension,Drugs,Alcohol,First_Appointment_Date,Last_Appointment_Date,DateOfDeath
0,1,1962-02-27,female,hispanic,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2013-04-27,2018-06-01,NaN
1,2,1959-08-18,male,white,0,0,0,0,0,0,...,0,0,0,0,1,0,0,2005-11-30,2008-11-02,2008-11-02
2,3,1946-02-15,female,white,0,0,0,0,0,0,...,0,1,0,0,1,0,0,2011-11-05,2015-11-13,NaN
3,4,1979-07-27,female,white,0,0,0,0,0,1,...,0,0,0,0,0,0,0,2010-03-01,2016-01-17,2016-01-17
4,5,1983-02-19,female,hispanic,0,0,0,0,0,0,...,0,0,0,0,1,0,0,2006-09-22,2018-06-01,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19996,1997-12-19,female,other,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2008-06-14,2018-06-01,NaN
19996,19997,1984-03-31,female,white,0,0,0,0,0,0,...,0,1,0,0,1,0,0,2007-04-24,2018-06-01,NaN
19997,19998,1993-07-04,female,white,0,0,0,0,0,0,...,0,0,1,0,1,0,0,2010-10-16,2018-06-01,NaN
19998,19999,1984-04-17,male,other,0,0,0,0,0,0,...,0,0,0,0,1,0,0,2015-01-04,2018-06-01,NaN


We need to make a variable to indicate mortality. We can do that based on the abscence of 'date of death':

In [6]:
# Create mortality variable
df_patient['mortality'] = \
    np.where(df_patient['DateOfDeath'].isnull(),
             0,1)
# Examine
df_patient['mortality']

,mortality
0,0
1,1
2,0
3,1
4,0
...,...
19995,0
19996,0
19997,0
19998,0


In [7]:
df_patient['mortality'].describe()

,mortality
count,20000.000000
mean,0.354700
std,0.478434
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [8]:
df_patient.describe()

,PatientID,Myocardial_infarction,Congestive_heart_failure,Peripheral_vascular_disease,Stroke,Dementia,Pulmonary,Rheumatic,Peptic_ulcer_disease,LiverMild,...,Cancer,LiverSevere,Metastatic_solid_tumour,HIV,Obesity,Depression,Hypertension,Drugs,Alcohol,mortality
count,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,...,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000
mean,10000.500000,0.045600,0.043450,0.023950,0.028650,0.031400,0.072650,0.012300,0.009650,0.009250,...,0.050450,0.051450,0.033150,0.006450,0.163450,0.106300,0.302900,0.040050,0.079750,0.354700
std,5773.647028,0.208621,0.203873,0.152897,0.166825,0.174401,0.259568,0.110224,0.097762,0.095733,...,0.218877,0.220919,0.179033,0.080054,0.369785,0.308229,0.459524,0.196081,0.270913,0.478434
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,5000.750000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,10000.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,15000.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000
max,20000.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [9]:
df_patient.dtypes

,0
PatientID,int64
DateOfBirth,object
Gender,object
Race,object
Myocardial_infarction,int64
...,...
Alcohol,int64
First_Appointment_Date,object
Last_Appointment_Date,object
DateOfDeath,object


We should change date of birth to be an actual date and calculate age if we want to include it in the model:

In [10]:
# Convert dateofBirth to date
df_patient['DateOfBirth'] = \
    pd.to_datetime(df_patient['DateOfBirth'])
# Calculate age in years as of 2015-01-01
df_patient['Age_years'] = \
    ((pd.to_datetime('2015-01-01') - df_patient['DateOfBirth']).dt.days/365.25)
df_patient['Age_years'].describe()

,Age_years
count,20000.000000
mean,47.247474
std,18.145086
min,15.753593
25%,31.733744
50%,47.099247
75%,62.924025
max,78.743326


Use Patsy to Create the Model Matrices
We typically start out with a pandas dataframe for manipulation purposes, then we will use this dataframe as the input to the machine learning library. I created a pandas dataframe above to replicate this process. We will use the dmatrices function from the patsy library to easily generate the design matrices for the machine learning algorithms representing the inputs. THis handles the following:

drops rows with missing data
construct one-hot encoding for categorical variables
optionally adds constant intecercept

In [11]:
df_patient.columns

Index(['PatientID', 'DateOfBirth', 'Gender', 'Race', 'Myocardial_infarction',
       'Congestive_heart_failure', 'Peripheral_vascular_disease', 'Stroke',
       'Dementia', 'Pulmonary', 'Rheumatic', 'Peptic_ulcer_disease',
       'LiverMild', 'Diabetes_without_complications',
       'Diabetes_with_complications', 'Paralysis', 'Renal', 'Cancer',
       'LiverSevere', 'Metastatic_solid_tumour', 'HIV', 'Obesity',
       'Depression', 'Hypertension', 'Drugs', 'Alcohol',
       'First_Appointment_Date', 'Last_Appointment_Date', 'DateOfDeath',
       'mortality', 'Age_years'],
      dtype='object')

In [12]:
## Create formula for all variables in model
vars_remove = ['PatientID','First_Appointment_Date','DateOfBirth',
               'Last_Appointment_Date','DateOfDeath','mortality']
vars_left = set(df_patient.columns) - set(vars_remove)
formula = "mortality ~ " + " + ".join(vars_left)
formula

'mortality ~ Pulmonary + Hypertension + LiverMild + Depression + Congestive_heart_failure + Stroke + Peptic_ulcer_disease + LiverSevere + Drugs + Paralysis + Renal + Cancer + Peripheral_vascular_disease + Dementia + Diabetes_with_complications + Metastatic_solid_tumour + Myocardial_infarction + Age_years + Obesity + Race + Rheumatic + Gender + HIV + Diabetes_without_complications + Alcohol'

Next, fit a series of logistic regression models, without regularization. Each model should use the same set of predictors (all of the relevant predictors in the dataset) and should use the entire dataset, rather than a fraction of it. Use a randomly chosen 80% proportion of observations for training and the remaining for checking the generalizable performance (i.e., performance on the holdout subset). Be sure to ensure that the training and holdout subsets are identical across all models. Each model should choose a different solver.

In [13]:
Y, X = dmatrices(formula, df_patient)

In [14]:
Y

DesignMatrix with shape (20000, 1)
  mortality
          0
          1
          0
          1
          0
          0
          0
          0
          0
          0
          0
          0
          1
          0
          0
          0
          1
          1
          0
          0
          1
          1
          0
          0
          0
          0
          0
          0
          0
          0
  [19970 rows omitted]
  Terms:
    'mortality' (column 0)
  (to view full data, use np.asarray(this_obj))

In [15]:
X

DesignMatrix with shape (20000, 28)
  Columns:
    ['Intercept',
     'Race[T.hispanic]',
     'Race[T.other]',
     'Race[T.white]',
     'Gender[T.male]',
     'Pulmonary',
     'Hypertension',
     'LiverMild',
     'Depression',
     'Congestive_heart_failure',
     'Stroke',
     'Peptic_ulcer_disease',
     'LiverSevere',
     'Drugs',
     'Paralysis',
     'Renal',
     'Cancer',
     'Peripheral_vascular_disease',
     'Dementia',
     'Diabetes_with_complications',
     'Metastatic_solid_tumour',
     'Myocardial_infarction',
     'Age_years',
     'Obesity',
     'Rheumatic',
     'HIV',
     'Diabetes_without_complications',
     'Alcohol']
  Terms:
    'Intercept' (column 0)
    'Race' (columns 1:4)
    'Gender' (column 4)
    'Pulmonary' (column 5)
    'Hypertension' (column 6)
    'LiverMild' (column 7)
    'Depression' (column 8)
    'Congestive_heart_failure' (column 9)
    'Stroke' (column 10)
    'Peptic_ulcer_disease' (column 11)
    'LiverSevere' (column 12)
    'D

In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, np.ravel(Y), test_size=0.2, random_state=42)

Confirm the Output Dimensions
We can confirm the dimensions of the data are the same within test and train
The proportion should also be close to the test_size argument.

In [17]:
## Confirm dimensions
X_train.shape

(16000, 28)

In [18]:
X_test.shape

(4000, 28)

In [19]:
y_train.shape

(16000,)

In [20]:
y_test.shape

(4000,)

Logistic Regression Models with Different Solvers that are not generalized

In [21]:
#solvers = ['lbfgs', 'newton-cg', 'newton-cholesky', 'sag', 'saga']

In [22]:
## Import the linear model
from sklearn import linear_model

## Define logistic regression model without regularization
clf = linear_model.LogisticRegression(
    fit_intercept=True,       # Include intercept
    penalty= None,           # No regularization
    solver='lbfgs',           # Solver (change to 'sag', 'saga', etc. as needed)
    max_iter=1000,            # Increase if convergence warning appears
    random_state=42           # For reproducibility
)

## Fit the model on the training data
clf.fit(X_train, y_train)

LogisticRegression(max_iter=1000, penalty=None, random_state=42)

In [23]:
## Make predictions on training dataset
## training error?
clf.predict(X_train)

array([1., 0., 0., ..., 1., 0., 0.])

In [24]:
## can also predict probabilities
clf.predict_proba(X_train)

array([[0.31351944, 0.68648056],
       [0.80849815, 0.19150185],
       [0.75084157, 0.24915843],
       ...,
       [0.45276665, 0.54723335],
       [0.93735455, 0.06264545],
       [0.95137493, 0.04862507]])

Model Summaries
We can extract the model coefficients with the .coef_ attribute.

In [25]:
## Get coefficients
clf.coef_
clf.coef_.shape

array([[-2.12761512, -0.0678081 , -0.10178332, -0.00693388,  0.02379821,
         0.63454209, -0.03253851,  0.14295405,  0.50467306,  0.46923544,
        -0.11179912, -0.03675728,  0.98011387,  0.33354558,  0.14045833,
        -0.06469803,  0.64904519,  1.14293271,  0.01568177,  0.15352078,
         0.03164523,  0.86156162,  0.06506742,  0.30891841, -0.30082583,
         0.14004478, -0.00219099,  0.79251471]])

(1, 28)

Assessing the Model Score (accuracy)
The concept of error is a bit different for a binary outcome than the continuous case.
We can construct error to be a function of the number of incorrect predictions.

In [26]:
## get mean accuracy
clf.score(X_train,y_train)

0.7479375

In this case, our model is accurate about 74% of the time! We can understand what that means by looking at the predictions against the actual outcomes. This is called a confusion matrix.

In [27]:
## get confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_train,
                 clf.predict(X_train))

array([[8747, 1597],
       [2436, 3220]])

In [28]:
## get classification metrics
print(sklearn.metrics.classification_report(y_train,
                                            clf.predict(X_train)))

              precision    recall  f1-score   support

         0.0       0.78      0.85      0.81     10344
         1.0       0.67      0.57      0.61      5656

    accuracy                           0.75     16000
   macro avg       0.73      0.71      0.71     16000
weighted avg       0.74      0.75      0.74     16000



In [29]:
## get accuracy
sklearn.metrics.accuracy_score(y_train,
                               clf.predict(X_train))

0.7479375

Keep Track of Scores Across Models
I am going to write a small function that will print the scores from a dict so we can compare the models. I will store the model scores in the dict as well.

In [30]:
## Create dict to store all these results:
result_scores = {}
## Score the Model on Training and Testing Set
result_scores['Logistic_Lbfgs'] = \
            (sklearn.metrics.accuracy_score(y_train,clf.predict(X_train)),
             sklearn.metrics.accuracy_score(y_test,clf.predict(X_test)))

In [31]:
## Create Function to Print Results
def get_results(x1):
    print("\n{0:20}   {1:4}    {2:4}".format('Model','Train','Test'))
    print('-------------------------------------------')
    for i in x1.keys():
        print("{0:20}   {1:<6.4}   {2:<6.4}".format(i,x1[i][0],x1[i][1]))

In [32]:
get_results(result_scores)


Model                  Train    Test
-------------------------------------------
Logistic_Lbfgs         0.7479   0.736 


Comparison to the Null Model
Is that a good score for accuracy? Compared to what? We can consider a null model of simply predicting the most frequent class as a base model. Without any other information, I may predict based simply on the distribution of the outcome.

In [33]:
## Null information rate
1 - y_train.mean()

array(0.6465)

Scikitlearn has a built in dummy classifier that works similarly:

In [34]:
## Dummy classifier
from sklearn.dummy import DummyClassifier
clf = DummyClassifier(strategy='most_frequent',
                      random_state=0)
clf.fit(X_train, y_train)
clf.score(X_train, y_train)

DummyClassifier(random_state=0, strategy='most_frequent')

0.6465

In [35]:
## Score the Model on Training and Testing Set
result_scores['Null'] = \
            (sklearn.metrics.accuracy_score(y_train,clf.predict(X_train)),
             sklearn.metrics.accuracy_score(y_test,clf.predict(X_test)))

In [36]:
get_results(result_scores)


Model                  Train    Test
-------------------------------------------
Logistic_Lbfgs         0.7479   0.736 
Null                   0.6465   0.6405


**Logistic Regression with solver newton-cg**

In [37]:
## Import the linear model
from sklearn import linear_model

## Define logistic regression model without regularization
clf = linear_model.LogisticRegression(
    fit_intercept=True,       # Include intercept
    penalty= None,           # No regularization
    solver='newton-cg',           # Solver (change to 'sag', 'saga', etc. as needed)
    max_iter=1000,            # Increase if convergence warning appears
    random_state=42           # For reproducibility
)

## Fit the model on the training data
clf.fit(X_train, y_train)

LogisticRegression(max_iter=1000, penalty=None, random_state=42,
                   solver='newton-cg')

In [38]:
## Make predictions on training dataset
## training error?
clf.predict(X_train)

array([1., 0., 0., ..., 1., 0., 0.])

In [39]:

## can also predict probabilities
clf.predict_proba(X_train)

array([[0.31379971, 0.68620029],
       [0.80891675, 0.19108325],
       [0.75111403, 0.24888597],
       ...,
       [0.45150574, 0.54849426],
       [0.93729544, 0.06270456],
       [0.95142321, 0.04857679]])

In [40]:
## Get coefficients
clf.coef_
clf.coef_.shape

array([[-2.12937344, -0.06532394, -0.09734054, -0.00501065,  0.0241333 ,
         0.63191925, -0.02853424,  0.14549623,  0.50486568,  0.47229923,
        -0.11224902, -0.03730295,  0.9797983 ,  0.33584051,  0.14027121,
        -0.0649341 ,  0.65104774,  1.13946491,  0.01593624,  0.15534209,
         0.03170931,  0.8644375 ,  0.06506685,  0.30907959, -0.30151925,
         0.14579801, -0.00213494,  0.79092065]])

(1, 28)

In [41]:

## get mean accuracy
clf.score(X_train,y_train)

0.7480625

In this case, our model is accurate about 74.8% of the time! We can understand what that means by looking at the predictions against the actual outcomes. This is called a confusion matrix.

In [42]:
## get confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_train,
                 clf.predict(X_train))

array([[8746, 1598],
       [2433, 3223]])

In [43]:
## get classification metrics
print(sklearn.metrics.classification_report(y_train,
                                            clf.predict(X_train)))

              precision    recall  f1-score   support

         0.0       0.78      0.85      0.81     10344
         1.0       0.67      0.57      0.62      5656

    accuracy                           0.75     16000
   macro avg       0.73      0.71      0.71     16000
weighted avg       0.74      0.75      0.74     16000



In [44]:
## get accuracy
sklearn.metrics.accuracy_score(y_train,
                               clf.predict(X_train))

0.7480625

In [45]:
## Score the Model on Training and Testing Set
result_scores['Logistics_newton-cg'] = \
            (sklearn.metrics.accuracy_score(y_train,clf.predict(X_train)),
             sklearn.metrics.accuracy_score(y_test,clf.predict(X_test)))

In [46]:
get_results(result_scores)


Model                  Train    Test
-------------------------------------------
Logistic_Lbfgs         0.7479   0.736 
Null                   0.6465   0.6405
Logistics_newton-cg    0.7481   0.7355


**Logistic Regression with solver newton-cholesky**

In [47]:
## Import the linear model
from sklearn import linear_model

## Define logistic regression model without regularization
clf = linear_model.LogisticRegression(
    fit_intercept=True,       # Include intercept
    penalty= None,           # No regularization
    solver='newton-cholesky',           # Solver (change to 'sag', 'saga', etc. as needed)
    max_iter=1000,            # Increase if convergence warning appears
    random_state=42           # For reproducibility
)

## Fit the model on the training data
clf.fit(X_train, y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_glm/_newton_solver.py:576: LinAlgWarning: The inner solver of NewtonCholeskySolver stumbled upon a singular or very ill-conditioned Hessian matrix at iteration 2. It will now resort to lbfgs instead.
Further options are to use another solver or to avoid such situation in the first place. Possible remedies are removing collinear features of X or increasing the penalization strengths.
The original Linear Algebra message was:
Ill-conditioned matrix (rcond=5.3802e-17): result may not be accurate.
  warnings.warn(


LogisticRegression(max_iter=1000, penalty=None, random_state=42,
                   solver='newton-cholesky')

In [48]:
## Make predictions on training dataset
## training error?
clf.predict(X_train)

array([1., 0., 0., ..., 1., 0., 0.])

In [49]:
## can also predict probabilities
clf.predict_proba(X_train)

array([[0.31358571, 0.68641429],
       [0.80843855, 0.19156145],
       [0.75136856, 0.24863144],
       ...,
       [0.45112475, 0.54887525],
       [0.93740793, 0.06259207],
       [0.9513935 , 0.0486065 ]])

In [50]:
## Get coefficients
clf.coef_
clf.coef_.shape

array([[-2.04525102e+00, -6.15268142e-02, -9.16455864e-02,
        -1.67788072e-03,  2.50204032e-02,  6.33325377e-01,
        -2.97633507e-02,  1.35648379e-01,  5.06440677e-01,
         4.72683266e-01, -1.13103007e-01, -4.18166356e-02,
         9.76494219e-01,  3.37089853e-01,  1.27653959e-01,
        -6.54281726e-02,  6.49148093e-01,  1.13090304e+00,
         1.58173076e-02,  1.56638683e-01,  3.19066619e-02,
         8.62864060e-01,  6.50655897e-02,  3.07568503e-01,
        -2.81839757e-01,  1.56958701e-01, -9.90455927e-04,
         7.94463871e-01]])

(1, 28)

In [51]:
## get mean accuracy
clf.score(X_train,y_train)

0.7480625

In [52]:

## get confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_train,
                 clf.predict(X_train))

array([[8747, 1597],
       [2434, 3222]])

In [53]:
## get classification metrics
print(sklearn.metrics.classification_report(y_train,
                                            clf.predict(X_train)))

              precision    recall  f1-score   support

         0.0       0.78      0.85      0.81     10344
         1.0       0.67      0.57      0.62      5656

    accuracy                           0.75     16000
   macro avg       0.73      0.71      0.71     16000
weighted avg       0.74      0.75      0.74     16000



In [54]:
## get accuracy
sklearn.metrics.accuracy_score(y_train,
                               clf.predict(X_train))

0.7480625

In [55]:
## Score the Model on Training and Testing Set
result_scores['Logistics_newton-cholesky'] = \
            (sklearn.metrics.accuracy_score(y_train,clf.predict(X_train)),
             sklearn.metrics.accuracy_score(y_test,clf.predict(X_test)))

In [56]:
get_results(result_scores)


Model                  Train    Test
-------------------------------------------
Logistic_Lbfgs         0.7479   0.736 
Null                   0.6465   0.6405
Logistics_newton-cg    0.7481   0.7355
Logistics_newton-cholesky   0.7481   0.736 


**Logistic Regression Models with Solvers 'sag'**

In [57]:

## Import the linear model
from sklearn import linear_model

## Define logistic regression model without regularization
clf = linear_model.LogisticRegression(
    fit_intercept=True,       # Include intercept
    penalty= None,           # No regularization
    solver='sag',           # Solver (change to 'sag', 'saga', etc. as needed)
    max_iter=1000,            # Increase if convergence warning appears
    random_state=42           # For reproducibility
)

## Fit the model on the training data
clf.fit(X_train, y_train)

LogisticRegression(max_iter=1000, penalty=None, random_state=42, solver='sag')

In [58]:
## Make predictions on training dataset
## training error?
clf.predict(X_train)

array([1., 0., 0., ..., 1., 0., 0.])

In [59]:

## can also predict probabilities
clf.predict_proba(X_train)

array([[0.31371269, 0.68628731],
       [0.80831296, 0.19168704],
       [0.75112003, 0.24887997],
       ...,
       [0.45193694, 0.54806306],
       [0.93738256, 0.06261744],
       [0.9513678 , 0.0486322 ]])

In [60]:

## Get coefficients
clf.coef_
clf.coef_.shape

array([[-2.12852768e+00, -6.53635201e-02, -9.49478983e-02,
        -5.00053315e-03,  2.45243135e-02,  6.34300449e-01,
        -3.08076970e-02,  1.48057713e-01,  5.03927577e-01,
         4.71500644e-01, -1.12685802e-01, -3.79979201e-02,
         9.77891674e-01,  3.35839997e-01,  1.34802891e-01,
        -6.50843528e-02,  6.49481299e-01,  1.12769213e+00,
         1.57102409e-02,  1.55300085e-01,  3.15317949e-02,
         8.62656079e-01,  6.50449775e-02,  3.08304152e-01,
        -2.96576022e-01,  1.64879548e-01, -1.04836823e-03,
         7.94075457e-01]])

(1, 28)

In [61]:
## get mean accuracy
clf.score(X_train,y_train)


0.7479375

In [62]:
## get confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_train,
                 clf.predict(X_train))

array([[8747, 1597],
       [2436, 3220]])

In [63]:

## get classification metrics
print(sklearn.metrics.classification_report(y_train,
                                            clf.predict(X_train)))

              precision    recall  f1-score   support

         0.0       0.78      0.85      0.81     10344
         1.0       0.67      0.57      0.61      5656

    accuracy                           0.75     16000
   macro avg       0.73      0.71      0.71     16000
weighted avg       0.74      0.75      0.74     16000



In [64]:

## get accuracy
sklearn.metrics.accuracy_score(y_train,
                               clf.predict(X_train))

0.7479375

In [65]:

## Score the Model on Training and Testing Set
result_scores['Logistics_sag'] = \
            (sklearn.metrics.accuracy_score(y_train,clf.predict(X_train)),
             sklearn.metrics.accuracy_score(y_test,clf.predict(X_test)))

In [66]:
get_results(result_scores)



Model                  Train    Test
-------------------------------------------
Logistic_Lbfgs         0.7479   0.736 
Null                   0.6465   0.6405
Logistics_newton-cg    0.7481   0.7355
Logistics_newton-cholesky   0.7481   0.736 
Logistics_sag          0.7479   0.7358


**Logistic Regression Models with Solver 'saga'**

In [67]:

## Import the linear model
from sklearn import linear_model

## Define logistic regression model without regularization
clf = linear_model.LogisticRegression(
    fit_intercept=True,       # Include intercept
    penalty= None,           # No regularization
    solver='saga',           # Solver (change to 'sag', 'saga', etc. as needed)
    max_iter=1000,            # Increase if convergence warning appears
    random_state=42           # For reproducibility
)

## Fit the model on the training data
clf.fit(X_train, y_train)

LogisticRegression(max_iter=1000, penalty=None, random_state=42, solver='saga')

In [68]:
## Make predictions on training dataset
## training error?
clf.predict(X_train)

array([1., 0., 0., ..., 1., 0., 0.])

In [69]:
## can also predict probabilities
clf.predict_proba(X_train)

array([[0.31369434, 0.68630566],
       [0.80828448, 0.19171552],
       [0.75109651, 0.24890349],
       ...,
       [0.45187105, 0.54812895],
       [0.93734829, 0.06265171],
       [0.95134998, 0.04865002]])

In [70]:

## Get coefficients
clf.coef_
clf.coef_.shape

array([[-2.12824062e+00, -6.54610976e-02, -9.51129373e-02,
        -5.15945209e-03,  2.45349216e-02,  6.34194538e-01,
        -3.07924299e-02,  1.38786936e-01,  5.03982618e-01,
         4.71434925e-01, -1.12585572e-01, -3.67126811e-02,
         9.77970041e-01,  3.35764012e-01,  1.31718937e-01,
        -6.51198317e-02,  6.49265506e-01,  1.12614745e+00,
         1.55052471e-02,  1.55359862e-01,  3.15901289e-02,
         8.62480006e-01,  6.50405654e-02,  3.08208003e-01,
        -2.89537385e-01,  1.44935439e-01, -1.05428189e-03,
         7.94044601e-01]])

(1, 28)

In [71]:

## get mean accuracy
clf.score(X_train,y_train)

0.748

In [72]:

## get confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_train,
                 clf.predict(X_train))

array([[8748, 1596],
       [2436, 3220]])

In [73]:

## get classification metrics
print(sklearn.metrics.classification_report(y_train,
                                            clf.predict(X_train)))

              precision    recall  f1-score   support

         0.0       0.78      0.85      0.81     10344
         1.0       0.67      0.57      0.61      5656

    accuracy                           0.75     16000
   macro avg       0.73      0.71      0.71     16000
weighted avg       0.74      0.75      0.74     16000



In [74]:

## get accuracy
sklearn.metrics.accuracy_score(y_train,
                               clf.predict(X_train))


0.748

In [75]:
## Score the Model on Training and Testing Set
result_scores['Logistics_saga'] = \
            (sklearn.metrics.accuracy_score(y_train,clf.predict(X_train)),
             sklearn.metrics.accuracy_score(y_test,clf.predict(X_test)))

In [76]:
get_results(result_scores)


Model                  Train    Test
-------------------------------------------
Logistic_Lbfgs         0.7479   0.736 
Null                   0.6465   0.6405
Logistics_newton-cg    0.7481   0.7355
Logistics_newton-cholesky   0.7481   0.736 
Logistics_sag          0.7479   0.7358
Logistics_saga         0.748    0.736 
